In [ ]:
# dependencies import
from common_dependencies import *

In [ ]:
def draw_map(data_map):
    with plt.style.context('dark_background'):
            fig, ax = plt.subplots()
        
            fig.set_figwidth(18)
            fig.set_figheight(8)
            fig.patch.set_alpha(0.0)
        
            ax.invert_yaxis()
            ax.xaxis.tick_top()
            ax.xaxis.set_label_position('top')
            ax.set_title("Предсказанная развернутая карта дефектов", fontsize=25)
            ax.set_xlabel('Номер датчика', fontsize=20)
            ax.set_ylabel('Номер измерения', fontsize=20)
            ax.tick_params(axis='both', labelsize = 20)

            map = ax.pcolormesh(data_map)
            cbar = fig.colorbar(map)
            cbar.ax.tick_params(labelsize=20)

            ax.xaxis.set_major_locator(ticker.MultipleLocator(60))
    
    plt.show()

In [ ]:
# paths for files with original data
PATH_TO_DATA = {
    'run_1': 
        ('data/Original data/run_1/run1_WM32_data.csv',
        'data/Original data/run_1/run1_WM32_defects.csv',
        'data/Original data/run_1/run1_WM32_pipe.csv'),
    'run_2':
        ('data/Original data/run_2/run2_WM32_data.csv',
        'data/Original data/run_2/run2_WM32_defects.csv',
        'data/Original data/run_2/run2_WM32_pipe.csv')
}

PATH_TO_MODEL = 'networks/CNN'

CROP_SIZE = 16
CROP_STEP = 1
RUN = 1
MODEL_VER = '02'
MODEL_NUM = '02'
XSHIFT = 200

for name in os.listdir(PATH_TO_MODEL):
    res = re.match(F'(id=v{MODEL_VER}n{MODEL_NUM}).*', name)
    if not res is None:
        PATH_TO_MODEL += '/' + res[0]
        break

# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

In [ ]:
# чтение данных
data_cl = pidf.PipeDataCsvWorker(*PATH_TO_DATA[f'run_{RUN}'])

In [ ]:
# Загрузка модели
load_model = keras.models.load_model(PATH_TO_MODEL)

In [ ]:
def make_pred_fast(model, cl: pidf.PipeDataCsvWorker, crop_size: int, crop_step: int):

    cl.roll_dfs_along_axis(XSHIFT, axis=1)
    
    ref_orig_df = cl.get_defects_df()
    or_rows = cl.get_data_df().shape[0]
    or_cols = cl.get_data_df().shape[1]

    cl.extend_dfs_for_prediction(crop_size, crop_step)

    ref_extend_df = cl.get_defects_df()
    ex_rows = cl.get_data_df().shape[0]
    ex_cols = cl.get_data_df().shape[1]
    
    x_data = cl.get_data_df()

    (x_data_time,
    x_data_amp) = dp.reshape_X_df_to_image_like_numpy(x_data, crop_size, crop_step)

    x_data_time = dp.standartize_data(x_data_time)
    x_data_amp = dp.standartize_data(x_data_amp)

    res = model.predict([x_data_time, x_data_amp])
    res = res[:,0]

    extend_arr = np.ones((ex_rows, ex_cols))     
    
    res_it = iter(res) 
    
    for j in range(0,  ex_cols - crop_size + 1, crop_step):
        for i in range(0, ex_rows - crop_size + 1, crop_step):  
            temp_add = next(res_it)
            if temp_add > 1:
                print(temp_add)
            if temp_add >= 0.5:
                extend_arr[i:i+crop_size, j:j+crop_size] += temp_add
            else:
                extend_arr[i:i+crop_size, j:j+crop_size] -= temp_add 
    

    left_up_border = crop_size - 1
    
    orig_arr = extend_arr[left_up_border:, left_up_border:][:or_rows, :or_cols]

    extend_df = pd.DataFrame(data=extend_arr, 
                             columns=cl.get_data_df().columns, 
                             index=cl.get_data_df().index)

    orig_df = pd.DataFrame(data=orig_arr, 
                       columns=cl.get_data_df().columns.tolist()[left_up_border:][:or_cols], 
                       index=cl.get_data_df().index.tolist()[left_up_border:][:or_rows])
    
    return orig_df, extend_df, ref_orig_df, ref_extend_df, orig_arr, extend_arr 

In [ ]:
orig_df, extend_df, ref_orig_df, ref_extend_df, orig_arr, extend_arr  = make_pred_fast(load_model, data_cl, CROP_SIZE, CROP_STEP)

In [ ]:
draw_map(ref_orig_df)
draw_map(orig_df)

In [ ]:
draw_map(ref_extend_df)
draw_map(extend_df)

In [ ]:
path = f'data/Drawing data/Run{RUN}/' + \
    f'model_id=v{MODEL_VER}n{MODEL_NUM}_crop(size={CROP_SIZE},step={CROP_STEP})_shift(x={XSHIFT},y=0).xlsx'

with pd.ExcelWriter(path) as writer:  
    orig_df.to_excel(writer, sheet_name='orig_res')
    extend_df.to_excel(writer, sheet_name='extend_res')
    
    ref_orig_df.to_excel(writer, sheet_name='orig_reference')
    ref_extend_df.to_excel(writer, sheet_name='extend_reference')